In [1]:
import time
import glob
import random

import torch
import torch.nn as nn
import neptune
from neptune.utils import stringify_unsupported

import numpy as np
import awkward as ak
import matplotlib.pyplot as plt

import uproot
# import ROOT

print('CPU count: ', torch.multiprocessing.cpu_count())

# torch.set_num_threads(40)

CPU count:  76


In [11]:
use_neptune = False
debug = True

In [3]:
if use_neptune:
    run = neptune.init_run(
        project="alikaan.guven/ParT",
        api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJhNDNjMWJhNS0wMDExLTQ2NzYtOWVjNS1lNzAzOWU4Mzc0MGMifQ==",
    )  # your credentials

param = {
    "input_dim": 5,
    "input_svdim": 5,
    "num_classes": 2,
    "pair_input_dim": 4,
    "embed_dims": [32, 128, 32],
    "for_inference": True,
    "lr": 1e-4

}

if use_neptune:
    run["parameters"] = stringify_unsupported(param)

In [4]:
# it = uproot.iterate(("/scratch-cbe/users/alikaan.gueven/ML_KAAN/predict/stop_M1000_988_ct200_2018/outputs/out_NANOAODSIMoutput_2_Skim.root:Events"), step_size=20)
# x = next(it)

In [5]:
# x["Jet_pt"][:,0]

In [6]:
import ParT

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# model = ParT.ParticleTransformerTagger(pf_input_dim=5,
#                                        sv_input_dim=5,
#                                        num_classes=2,
#                                        pair_input_dim=4,
#                                        # embed_dims=[32, 128, 32],
#                                        for_inference=True).to(device, dtype=float)

model = ParT.ParticleTransformerDVTagger(input_dim=param['input_dim'],
                                         input_svdim=param['input_svdim'],
                                         num_classes=param['num_classes'],
                                         pair_input_dim=param['pair_input_dim'],
                                         embed_dims=param['embed_dims'],
                                         for_inference=param['for_inference']).to(device, dtype=float)

                                                              
optimizer = torch.optim.Adam(model.parameters(), lr=param['lr'])
criterion = nn.CrossEntropyLoss()


num_parameters = sum(parameter.numel() for parameter in model.parameters())
if use_neptune:
    run['num_parameters'] = num_parameters

print(model)
print('num parameters: ', num_parameters)

/groups/hephy/cms/alikaan.gueven/conda/envs/coffea_torch/lib/python3.11/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


ParticleTransformerDVTagger(
  (trimmer): SequenceTrimmer()
  (embed): Embed(
    (input_bn): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (embed): Sequential(
      (0): LayerNorm((5,), eps=1e-05, elementwise_affine=True)
      (1): Linear(in_features=5, out_features=32, bias=True)
      (2): GELU(approximate='none')
      (3): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      (4): Linear(in_features=32, out_features=128, bias=True)
      (5): GELU(approximate='none')
      (6): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (7): Linear(in_features=128, out_features=32, bias=True)
      (8): GELU(approximate='none')
    )
  )
  (pair_embed): PairEmbed(
    (embed): Sequential(
      (0): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Conv1d(4, 64, kernel_size=(1,), stride=(1,))
      (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): GELU(

In [8]:
import MTUprootDataLoaderSimplified as MTUprootDataLoader

In [9]:
ModifiedUprootIterator = MTUprootDataLoader.ModifiedUprootIterator

In [10]:
MLDATADIR = '/scratch-cbe/users/alikaan.gueven/ML_KAAN'
tmpList = glob.glob(f'{MLDATADIR}/predict/stop_M1000_*/**/*.root', recursive=True)
random.shuffle(tmpList)
tmpList = [file + ':Events' for file in tmpList]
maxTrain = round(len(tmpList)*0.70)
maxVal = round(len(tmpList)*0.80)
trainList = tmpList[:maxTrain]
valList = tmpList[maxTrain:maxVal]
testList = tmpList[maxVal:]

branchDict = {}
branchDict['aux'] = ['nSDVSecVtx']
branchDict['ev'] = ['MET_phi']
branchDict['jet'] = ['Jet_phi', 'Jet_pt', 'Jet_eta']
branchDict['sv'] = ['SDVSecVtx_Lxy', 'SDVSecVtx_LxySig', 'SDVSecVtx_pAngle', 'SDVSecVtx_charge', 'SDVSecVtx_ndof',
                    'SDVIdxLUT_SecVtxIdx', 'SDVIdxLUT_TrackIdx']
branchDict['tk'] = ['SDVTrack_pt', 'SDVTrack_dxy', 'SDVTrack_dxyError', 'SDVTrack_dz', 'SDVTrack_dzError',
                    'SDVTrack_normalizedChi2', 'SDVTrack_eta', 'SDVTrack_phi']
branchDict['label'] = ['SDVSecVtx_matchedLLPnDau_bydau']

shuffle = True
nWorkers = 8
step_size = 256

trainDataset = ModifiedUprootIterator(trainList, branchDict, shuffle=shuffle, nWorkers=nWorkers, step_size=step_size)
trainLoader = torch.utils.data.DataLoader(trainDataset, num_workers=nWorkers, prefetch_factor=1, persistent_workers= True)

valDataset = ModifiedUprootIterator(valList, branchDict, shuffle=shuffle, nWorkers=nWorkers, step_size=step_size)
valLoader = torch.utils.data.DataLoader(valDataset, num_workers=nWorkers, prefetch_factor=1, persistent_workers= True)

testDataset = ModifiedUprootIterator(testList, branchDict, shuffle=shuffle, nWorkers=nWorkers, step_size=step_size)
testLoader = torch.utils.data.DataLoader(testDataset, num_workers=nWorkers, prefetch_factor=1, persistent_workers= True)


Initialize iterable dataset
Initialize iterable dataset
Initialize iterable dataset


In [12]:
if debug:
    shuffle = False
    nWorkers = 1
    step_size = 100
else:
    shuffle = True
    nWorkers = 8
    step_size = 256


trainDataset = ModifiedUprootIterator(trainList, branchDict, shuffle=shuffle, nWorkers=nWorkers, step_size=step_size)
trainLoader = torch.utils.data.DataLoader(trainDataset, num_workers=nWorkers, prefetch_factor=1, persistent_workers= True)



epochs = 15

model.train()
for epoch in range(epochs):
    start_time = time.time()
    losses = []
    TP_batch = []
    all_batch = []
    
    for batch_num, input_data in enumerate(trainLoader):
        optimizer.zero_grad()
        X = input_data

        tk_pair_features = torch.cat((X['SDVTrack_px'][0],
                                      X['SDVTrack_py'][0],
                                      X['SDVTrack_pz'][0],
                                      X['SDVTrack_E'][0],), dim=1)

        tk_features = torch.cat((X['SDVTrack_dxy'][0],
                                 X['SDVTrack_dxyError'][0],
                                 X['SDVTrack_dz'][0],
                                 X['SDVTrack_dzError'][0],
                                 X['SDVTrack_normalizedChi2'][0]), dim=1)
        
        sv_features = torch.cat((X['SDVSecVtx_Lxy'][0],
                                 X['SDVSecVtx_LxySig'][0],
                                 X['SDVSecVtx_pAngle'][0],
                                 X['SDVSecVtx_charge'][0],
                                 X['SDVSecVtx_ndof'][0]), dim=1)[..., np.newaxis]



        label = X['SDVSecVtx_matchedLLPnDau_bydau'][0]
        y = label
                
        
        tk_pair_features = tk_pair_features.to(device, dtype=float)
        tk_features = tk_features.to(device, dtype=float)
        sv_features = sv_features.to(device, dtype=float)
        
        y = y.to(device, dtype=float)

        ymaxSig = torch.max(y > 1, axis=-1).values
        ymaxSig = ymaxSig.float()
        yBkg = (ymaxSig != 1).float()
        y = torch.concatenate((yBkg[:, np.newaxis], ymaxSig[:, np.newaxis]), axis=-1)
        
        output = model(x=tk_features,
                       v=tk_pair_features,
                       x_sv=sv_features)
        loss = criterion(output, y)
        loss.backward()
        losses.append(loss.item())

        optimizer.step()
        # print('output: ', output.shape)
        # print('y: ', y.shape)
        # print(output>1)
        # break
        
        output_mask = sv_features[:,0,0] == 0 # [batch_dim, Lxy, 1-dim]
        acc = (torch.sum((output[~output_mask]>0.5) == y[~output_mask].data)) / (2*len(y[~output_mask])) # 2 is the number of classes
        if use_neptune:
            run["train/accuracy_batch"].append(acc)
            run["train/loss_batch"].append(loss.item())

        TP_batch.append((torch.sum(~output_mask) * acc).item())
        all_batch.append(torch.sum(~output_mask).item())
        
        # if batch_num % 6 == 0:
            # pass
            # output_mask = sv_features[:,0,0] == 0 # [batch_dim, Lxy, 1-dim]
            # acc = (torch.sum((output[~output_mask]>0.5) == y[~output_mask].data)) / (2*len(y[~output_mask])) # 2 is the number of classes
            # run["accuracy"].append(acc)
            # run["loss"].append(loss.item())
            # if acc != 1.:
                # print('\tEpoch %d | Batch %d | Loss %6.2f | 1/(1-Accu) %6.2f' % (epoch, batch_num, loss.item(), 1/(1-acc)))
                # print('\tEpoch %d | Batch %d | Loss %6.2f | Acc %6.2f' % (epoch, batch_num, loss.item(), acc))
            # else:
                # print('\tEpoch %d | Batch %d | Loss %6.2f | 1/(1-Accu) inf' % (epoch, batch_num, loss.item()))

    if use_neptune:
        run["train/accuracy_epoch"].append(sum(TP_batch) / sum(all_batch))
        run["train/losses_epoch"].append(sum(losses)/len(losses))


    
    ###### Inference on TEST ######
    with torch.no_grad():
        losses = []
        TP_batch = []
        all_batch = []
        for batch_num, input_data in enumerate(valLoader):
            X = input_data

            tk_pair_features = torch.cat((X['SDVTrack_px'][0],
                                          X['SDVTrack_py'][0],
                                          X['SDVTrack_pz'][0],
                                          X['SDVTrack_E'][0],), dim=1)
    
            tk_features = torch.cat((X['SDVTrack_dxy'][0],
                                     X['SDVTrack_dxyError'][0],
                                     X['SDVTrack_dz'][0],
                                     X['SDVTrack_dzError'][0],
                                     X['SDVTrack_normalizedChi2'][0]), dim=1)
            
            sv_features = torch.cat((X['SDVSecVtx_Lxy'][0],
                                     X['SDVSecVtx_LxySig'][0],
                                     X['SDVSecVtx_pAngle'][0],
                                     X['SDVSecVtx_charge'][0],
                                     X['SDVSecVtx_ndof'][0]), dim=1)[..., np.newaxis]
    
    
    
            label = X['SDVSecVtx_matchedLLPnDau_bydau'][0]
            y = label
    
            tk_pair_features = tk_pair_features.to(device, dtype=float)
            tk_features = tk_features.to(device, dtype=float)
            sv_features = sv_features.to(device, dtype=float)
            
            y = y.to(device, dtype=float)
    
            ymaxSig = torch.max(y > 1, axis=-1).values
            ymaxSig = ymaxSig.float()
            yBkg = (ymaxSig != 1).float()
            y = torch.concatenate((yBkg[:, np.newaxis], ymaxSig[:, np.newaxis]), axis=-1)
            
            output = model(x=tk_features,
                           v=tk_pair_features,
                           x_sv=sv_features)
            loss = criterion(output, y)
            losses.append(loss.item())
            
            output_mask = sv_features[:,0,0] == 0 # [batch_dim, Lxy, 1-dim]
            acc = (torch.sum((output[~output_mask]>0.5) == y[~output_mask].data)) / (2*len(y[~output_mask])) # 2 is the number of classes
            if use_neptune:
                run["val/accuracy_batch"].append(acc)
                run["val/loss_batch"].append(loss.item())
    
            TP_batch.append((torch.sum(~output_mask) * acc).item())
            all_batch.append(torch.sum(~output_mask).item())

        if use_neptune:
            run["val/accuracy_epoch"].append(sum(TP_batch) / sum(all_batch))
            run["val/losses_epoch"].append(sum(losses)/len(losses))
            
    
    
    # if len(losses) > 0:
    #     print('Epoch %d | Loss %6.2f' % (epoch, sum(losses)/len(losses)))
    # else:
    #     print('Epoch %d ' % (epoch))
    end_time = time.time()
    print(f"Time spent for epoch: {end_time - start_time:.2f} s.")

Initialize iterable dataset


KeyError: 'SDVSecVtx_matchedLLPnDau_bydau'

In [ ]:
if use_neptune:
    run.stop()

# Inference

## Test Dataset

In [ ]:
# model.eval()
# 
# correct_predictions = []
# all_predictions = []
# 
# 
# with torch.no_grad():
#     for (x,y) in testLoader:
#         
#         x = x.to(device, dtype=float)
#         y = y.to(device, dtype=float)
# 
#         x = x[0]
# 
#         ymaxSig = torch.max(y[0] > 1, axis=-1).values
#         ymaxSig = ymaxSig.float()
#         yBkg = (ymaxSig != 1).float()
#         y = torch.concatenate((yBkg, ymaxSig), axis=-1)
#         
#         pred = model(x).argmax(dim=-1)
#         summ = np.sum(np.asarray(pred == y.argmax(dim=-1), dtype=float))
#         size = pred.shape.numel()
#         print(f'Batch accuracy after training: {summ/size:.3f}')

## Train Dataset

In [ ]:
# model.eval()
# 
# with torch.no_grad():
#     for (x,y) in trainLoader:
#         
#         x = x.to(device, dtype=float)
#         y = y.to(device, dtype=float)
# 
#         x = x[0]
# 
#         ymaxSig = torch.max(y[0] > 1, axis=-1).values
#         ymaxSig = ymaxSig.float()
#         yBkg = (ymaxSig != 1).float()
#         y = torch.concatenate((yBkg, ymaxSig), axis=-1)
#         
#         pred = model(x).argmax(dim=-1)
#         summ = np.sum(np.asarray(pred == y.argmax(dim=-1), dtype=float))
#         size = pred.shape.numel()
#         print(f'Batch accuracy after training: {summ/size:.3f}')

In [ ]:
# pred = model(x)
# plt.hist(pred.detach().numpy()[:,1], bins=np.arange(0,1.01,0.01), histtype='step')
# plt.yscale('log')
# plt.ylim(1,1e3)
# plt.show()

In [ ]:
# plt.hist(y.detach().numpy()[:,1], bins=np.arange(0,1.01,0.01), histtype='step')
# plt.yscale('log')
# plt.ylim(1,1e3)
# plt.show()

In [ ]:
# np.sum(y.argmax(dim=-1).numpy() == 1)

In [ ]:
# np.sum(y.argmax(dim=-1).numpy() == 0)

In [ ]:
# 438 / (346 + 438)